<a href="https://colab.research.google.com/github/nedokormysh/Stepik_Ai_edu_practical_ML/blob/week_5_logistic_regression/LogReg_final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [1]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [3]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [4]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [6]:
# ваш код здесь
data.describe(include='object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [9]:
data.select_dtypes(include='object').columns

Index(['Marital Status', 'Gender', 'Education', 'Occupation', 'Home Owner',
       'Commute Distance', 'Region', 'Purchased Bike'],
      dtype='object')

In [14]:
# ваш код здесь

for l in data.select_dtypes(include='object').columns:
    # print(data[l].nunique())
    if data[l].nunique() == 2:
        # print(data[l].value_counts())
        # data['Marital Status'] = data['Marital Status'].map({'Married' : 1, 'Single' : 0})
        # data[l] = data[l].map({'Married' : 1, 'Single' : 0})
        data[l] = (data[l] == data[l].mode()[0]).astype(int)

In [20]:
data.head()

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,1,0,40000,1,Bachelors,Skilled Manual,1,0,0-1 Miles,Europe,42,1
1,24107,1,1,30000,3,Partial College,Clerical,1,1,0-1 Miles,Europe,43,1
2,14177,1,1,80000,5,Partial College,Professional,0,2,2-5 Miles,Europe,60,1
3,24381,0,1,70000,0,Bachelors,Professional,1,1,5-10 Miles,Pacific,41,0
4,25597,0,1,30000,0,Bachelors,Clerical,0,0,0-1 Miles,Europe,36,0


Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [ ]:
# another
# n_unique_values = data.describe(include="object").loc["unique"]
# binary_cat_features = n_unique_values.loc[n_unique_values == 2].index
# print(binary_cat_features)

# for column in binary_cat_features:
#     mode_value = data[column].mode().values[0]
#     data[column] = (data[column] == mode_value).astype(int)

# not_binary_cat_features = n_unique_values.loc[n_unique_values != 2].index
# print(not_binary_cat_features)

# data.drop(columns=not_binary_cat_features, inplace=True)
# print(len(not_binary_cat_features))

In [ ]:
# ваш код здесь
data.drop(data.select_dtypes(include='object').columns, axis=1, inplace=True)

Удалите столбец `ID`, так как он по сути нечисловой.

In [25]:
# ваш код здесь
data.drop('ID', axis=1, inplace=True)

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [26]:
# ваш код здесь
X = data.drop('Purchased Bike', axis=1)
y = data['Purchased Bike']

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [44]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [28]:
# ваш код здесь
Xtrain.shape

(700, 7)

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [45]:
from sklearn.preprocessing import MinMaxScaler

# ваш код здесь
sc = MinMaxScaler()

Xtrain = pd.DataFrame(sc.fit_transform(Xtrain))
Xtest = pd.DataFrame(sc.transform(Xtest))

Теперь обучите логистическую регрессию на тренировочных данных

In [39]:
from sklearn.linear_model import LogisticRegression

# ваш код здесь

model = LogisticRegression()

model.fit(Xtrain, ytrain)

LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [48]:
# ваш код здесь

Xtrain_preds = model.predict(Xtrain)
Xtest_preds = model.predict(Xtest)

In [49]:
Xtest_preds.shape

(300,)

Оцените значение accuracy на трейне и на тесте.

In [41]:
from sklearn.metrics import accuracy_score

# ваш код здесь
accuracy_score(Xtrain_preds, ytrain)

0.6342857142857142

In [42]:
# Xtest_preds

In [50]:
accuracy_score(Xtest_preds, ytest)

0.58

Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [51]:
from sklearn.preprocessing import PolynomialFeatures

# ваш код здесь
poly = PolynomialFeatures(degree=2)
Xtrain = pd.DataFrame(poly.fit_transform(Xtrain))
Xtest = pd.DataFrame(poly.transform(Xtest))

**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

In [52]:
Xtrain.shape

(700, 36)

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [53]:
# ваш код здесь
model.fit(Xtrain, ytrain)

LogisticRegression()

In [54]:
Xtrain_preds_poly = model.predict(Xtrain)
Xtest_preds_poly = model.predict(Xtest)

In [56]:
accuracy_score(Xtrain_preds_poly, ytrain)

0.6871428571428572

In [57]:
accuracy_score(Xtest_preds_poly, ytest)

0.6233333333333333

In [58]:
round(0.6233333333333333 - 0.58, 2)

0.04

**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [60]:
from sklearn.metrics import recall_score

In [61]:
# ваш код здесь
accuracy_score(Xtest_preds_poly, ytest), recall_score(Xtest_preds_poly, ytest)

(0.6233333333333333, 0.6023391812865497)

Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [62]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 36), (210, 36))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [64]:
model.fit(XtrainS, ytrainS)
predS = model.predict_proba(Xval)

In [71]:
# predS[:,1] >= 0.5

In [78]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1

for thr in np.arange(0, 1, 0.01):
    # ваш код здесь
    new_predictions = (predS[:, 1] > thr).astype(int)
    acc = accuracy_score(y_true=yval, y_pred=new_predictions)
    # print(acc)

    if acc >= 0.6:
        rec = recall_score(y_true=yval, y_pred=new_predictions)
        if rec > RecMax:
            RecMax = rec
            BestThr = thr
            BestAcc = acc

    # recall_score(y_true=yval, ytest=new_predictions)

print(BestThr, RecMax, BestAcc)

0.42 0.8518518518518519 0.6142857142857143


Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [79]:
# ваш код здесь
model.fit(Xtrain, ytrain)

LogisticRegression()

In [80]:
pr = model.predict_proba(Xtest)

In [81]:
newpr = (pr[:, 1] > 0.42).astype(int)

In [82]:
recall_score(y_true=ytest, y_pred=newpr)

0.8040540540540541

**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.